#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann 
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/
* https://kidger.site/thoughts/jaxtyping/
* https://github.com/patrick-kidger/torchtyping/tree/master

## Задачи для совместного разбора

In [2]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th

Scalar = TensorType[()]
patch_typeguard()

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат. 

In [29]:
class Neuron:
    def __init__(self, n_features: int, bias: float):
        # <создать атрибуты объекта weights и bias>
        self.weights: TensorType[n_features] = th.randn(n_features)
        print(self.weights)
        self.bias: float = bias

    def forward(self, inputs: TensorType["n_features"]) -> Scalar:
        return th.dot(inputs, self.weights) + self.bias # <реализовать логику нейрона>


In [15]:
import torch as th
inputs = th.tensor([1.0, 2.0, 3.0, 4.0])

In [50]:
neuron = Neuron(4, True)
neuron.forward(inputs)


tensor([ 0.8616,  0.9866, -0.0651,  0.4107])


tensor(5.2826)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [ ]:
class ReLU:
    @typechecked
    def forward(self, inputs: TensorType["n_features"]) -> TensorType["n_features"]:
         # <реализовать логику ReLU>
        return th.maximum(0, inputs)

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [52]:
class MSELoss:
    @typechecked
    def forward(self, y_pred: TensorType["batch"], y_true: TensorType["batch"]) -> Scalar:
        return (y_pred - y_true).square().mean()

In [53]:
y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])

In [55]:
l = MSELoss()
l = l.forward(y_pred, y_true)
l

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1"></p>

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения. 

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

- [ ] Проверено на семинаре

In [56]:
class Linear:
    def __init__(self, n_neurons: int, n_features: int, bias: bool = False) -> None:
        self.w = th.randn(n_features, n_neurons)
        self.b = th.randn(n_neurons)

    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "n_neurons"]:
        return inputs @ self.w + self.b
    #return th.mm(inputs, self.w) + self.b

<p class="task" id="2"></p>

2\. Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`. 

- [ ] Проверено на семинаре

In [ ]:
batch_size = 5
n_features = 4

l1 = Linear(10, n_features)
l2 = Linear(7, 10)

inputs = th.randn(batch_size, n_features)


In [58]:
y1 = l1.forward(inputs)
y2 = l2.forward(y1)
y2.size()

torch.Size([5, 7])

### Создание функций активации

<p class="task" id="3"></p>

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

- [ ] Проверено на семинаре

In [ ]:
class Softmax:
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
         # <реализовать логику Softmax>
        exps = th.exp(inputs - th.max(inputs, dim=-1, keepdim=True).values)
        return exps / th.sum(exps, dim=-1, keepdim=True)

<p class="task" id="4"></p>

4 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

- [ ] Проверено на семинаре

In [ ]:
class ELU:
    def __init__(self, alpha: float) -> None:
        self.alpha = alpha
    
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        # <реализовать логику ELU>
        return th.where(inputs > 0, inputs, self.alpha * (th.exp(inputs) - 1))

### Создание функции потерь

<p class="task" id="5"></p>

5 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$ 

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 2 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

- [ ] Проверено на семинаре

In [12]:
class CrossEntropyLoss:
    def forward(self, y_pred: TensorType["batch", float], y_true: TensorType["batch", int]):
        # <реализовать логику функции потерь>
        pass

<p class="task" id="6"></p>

6 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

- [ ] Проверено на семинаре

In [13]:
class MSERegularized:
    def __init__(self, lambda_: float) -> None:
        self.lambda_ = lambda_

    def data_loss(
            self, 
            y_pred: TensorType["batch"], 
            y_true: TensorType["batch"],
    ) -> Scalar:
        # <подсчет первого слагаемого из формулы>
        pass

    def reg_loss(self, weights: TensorType["batch", 1])  -> Scalar:
        # <подсчет второго слагаемого из формулы>
        pass

    def forward(
        self, 
        y_pred: TensorType["batch"], 
        y_true: TensorType["batch"], 
        weights: TensorType["batch", 1],
    ) -> Scalar:
        return self.data_loss(y_pred, y_true) + lambda_ * self.reg_loss(weights)